<a href="https://colab.research.google.com/github/NadiaHolmlund/M6_Group_Assignments/blob/main/Group_Assignment_3/Group_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

Select one of your previous projects that includes a machine learning component and use MLflow to track and manage your machine learning experiments. The following tasks should be performed:

- Train a machine learning model using the data from your previous project. You can use any machine learning model that is appropriate for your data and problem.

- Use MLflow to track and manage your machine learning experiments. Log the hyperparameters, metrics, and artifacts of your machine learning experiments in MLflow. Save structured and unstructured information related to your trained model in SQLite within MLflow.

# Imports

In [1]:
!pip install mlflow -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import sqlite3

pd.set_option('max_colwidth', 1000)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 1000]


# Cleaning the data

In [3]:
# Reading the CSV files into Pandas Dataframes and merging them together based on player ID
baseball_master = pd.read_csv('https://raw.githubusercontent.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_3/Data/Master.csv', encoding="ISO-8859-1")
baseball_batting = pd.read_csv('https://raw.githubusercontent.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_3/Data/Batting.csv', encoding="ISO-8859-1")

baseball = baseball_master.merge(baseball_batting, on = 'playerID')

In [4]:
# Examining the DataFrame
baseball.head()

,lahmanID,playerID,managerID,hofID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,1,aaronha01,NaN,aaronha01h,1934.0,2.0,5.0,USA,AL,Mobile,...,2.0,2.0,28.0,39.0,NaN,3.0,6.0,4.0,13.0,122.0
1,1,aaronha01,NaN,aaronha01h,1934.0,2.0,5.0,USA,AL,Mobile,...,3.0,1.0,49.0,61.0,5.0,3.0,7.0,4.0,20.0,153.0
2,1,aaronha01,NaN,aaronha01h,1934.0,2.0,5.0,USA,AL,Mobile,...,2.0,4.0,37.0,54.0,6.0,2.0,5.0,7.0,21.0,153.0
3,1,aaronha01,NaN,aaronha01h,1934.0,2.0,5.0,USA,AL,Mobile,...,1.0,1.0,57.0,58.0,15.0,0.0,0.0,3.0,13.0,151.0
4,1,aaronha01,NaN,aaronha01h,1934.0,2.0,5.0,USA,AL,Mobile,...,4.0,1.0,59.0,49.0,16.0,1.0,0.0,3.0,21.0,153.0


In [5]:
# Examining the DataFrame
baseball.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96609 entries, 0 to 96608
Data columns (total 56 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lahmanID      96609 non-null  int64  
 1   playerID      96609 non-null  object 
 2   managerID     6546 non-null   object 
 3   hofID         17793 non-null  object 
 4   birthYear     96294 non-null  float64
 5   birthMonth    95759 non-null  float64
 6   birthDay      95397 non-null  float64
 7   birthCountry  96174 non-null  object 
 8   birthState    86389 non-null  object 
 9   birthCity     95844 non-null  object 
 10  deathYear     39107 non-null  float64
 11  deathMonth    39096 non-null  float64
 12  deathDay      39095 non-null  float64
 13  deathCountry  38839 non-null  object 
 14  deathState    38415 non-null  object 
 15  deathCity     38804 non-null  object 
 16  nameFirst     96557 non-null  object 
 17  nameLast      96609 non-null  object 
 18  nameNote      2459 non-nul

In [6]:
# Extracting columns to be included in the database
baseball_db = baseball[['weight', 'height', 'G', 'AB', 'HR']]

In [7]:
# Dropping NaN values
baseball_db = baseball_db.dropna()

In [8]:
# Examining the dataFrame
baseball_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88718 entries, 0 to 96604
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   weight  88718 non-null  float64
 1   height  88718 non-null  float64
 2   G       88718 non-null  int64  
 3   AB      88718 non-null  float64
 4   HR      88718 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 4.1 MB


# Setting up the Data Layer

Creating a SQLite database for the baseball dataset

In [9]:
#database.py
import sqlite3
import pandas as pd

def init_db():
  # Connect to the SQLite database
  conn = sqlite3.connect("baseball.db")

  # Save the Pandas DataFrame to the SQLite database
  baseball_db.to_sql("baseball", conn, if_exists="replace", index=False)

  # Close the connection to the SQLite database
  conn.close()

if __name__ == '__main__':
    init_db()

# Setting up the Business Layer with MLFlow

Using the Scikit-Learn library to train a machine learning model for HR prediction.

The code also sets up an experiment named "HR_Prediction_exp_x" and logs the model's parameters, performance metrics, and the trained model itself as an artifact in MLflow.

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("baseball.db")

# Read data from a table using Pandas
data_df = pd.read_sql("SELECT * FROM baseball", conn)

def train_model():
    mlflow.set_experiment("HR_Prediction_exp_0")
    X = data_df.drop('HR', axis=1)
    y = data_df['HR']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    RFR = RandomForestRegressor(n_estimators=100, criterion="squared_error")

    with mlflow.start_run():
        RFR.fit(X_train, y_train)

        # Log model parameters
        mlflow.log_param("n_estimators", RFR.n_estimators)
        mlflow.log_param("criterion", RFR.criterion)

        # Log model performance metrics
        train_score = RFR.score(X_train, y_train)
        test_score = RFR.score(X_test, y_test)
        mlflow.log_metric("train_score", train_score)
        mlflow.log_metric("test_score", test_score)

        # Save the model as an artifact
        mlflow.sklearn.log_model(RFR, "model")

    return RFR, test_score

if __name__ == '__main__':
    RFR, accuracy = train_model()
    print(f"Model trained with accuracy: {accuracy}")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(RFR, "model")
    mlflow.sklearn.log_model(RFR, "model", registered_model_name="HR_model")
    mlflow.sklearn.save_model(RFR, "HR_model")

    # Launch MLflow UI
    import os
    os.system("mlflow ui")

2023/05/01 10:48:19 INFO mlflow.tracking.fluent: Experiment with name 'HR_Prediction_exp_0' does not exist. Creating a new experiment.


Model trained with accuracy: 0.6513884209925522


Successfully registered model 'HR_model'.
2023/05/01 10:48:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: HR_model, version 1
Created version '1' of model 'HR_model'.


# Experiments saved in MLFlow

## HR_Prediction_exp_0

![picture](https://raw.github.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_3/Data/HR_Prediction_exp_0.png)

## HR_Prediction_exp_1

![picture](https://raw.github.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_3/Data/HR_Prediction_exp_1.png)

## HR_Prediction_exp_2

![](https://raw.github.com/NadiaHolmlund/M6_Group_Assignments/main/Group_Assignment_3/Data/HR_Prediction_exp_2.png)